In [18]:
!pip install transformers datasets

In [19]:
from transformers import BertTokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import TrainingArguments, Trainer, BertForSequenceClassification
from transformers import BertModel

In [20]:
# Load dataset (you can replace this with your CSV)
dataset = load_dataset("imdb",split='train[:2000]')
dataset = dataset.train_test_split(test_size=0.2)
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
val_texts = dataset['test']['text']
val_labels = dataset['test']['label']

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [21]:
#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
def tokenize(batch):
    return tokenizer(batch['text'],padding=True, truncation = True, return_tensors='pt')

In [23]:
dataset = dataset.map(lambda x: tokenizer(x['text'], padding="max_length", truncation=True), batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [24]:
#Load pre-trained model with new output head
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
for param in model.bert.parameters():
    #param.requires_grad = False
    print(len(param))

30522
512
2
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768


In [26]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 109483778


In [27]:
trainable_params = sum(p.numel() for p in model.parameters () if p.requires_grad)
print(f"Trainable parameters : {trainable_params}")

Trainable parameters : 109483778


In [28]:
for name,param in model.named_parameters():
  print(f"{name}:50 | shape: {tuple(param.shape)} | Trainable: {param.requires_grad}")

bert.embeddings.word_embeddings.weight:50 | shape: (30522, 768) | Trainable: True
bert.embeddings.position_embeddings.weight:50 | shape: (512, 768) | Trainable: True
bert.embeddings.token_type_embeddings.weight:50 | shape: (2, 768) | Trainable: True
bert.embeddings.LayerNorm.weight:50 | shape: (768,) | Trainable: True
bert.embeddings.LayerNorm.bias:50 | shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.self.query.weight:50 | shape: (768, 768) | Trainable: True
bert.encoder.layer.0.attention.self.query.bias:50 | shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.self.key.weight:50 | shape: (768, 768) | Trainable: True
bert.encoder.layer.0.attention.self.key.bias:50 | shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.self.value.weight:50 | shape: (768, 768) | Trainable: True
bert.encoder.layer.0.attention.self.value.bias:50 | shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.output.dense.weight:50 | shape: (768, 768) | Trainable: True
be

In [29]:
print(model.bert.embeddings)
print(model.bert.encoder.layer[11])
print(model.classifier)

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): 

In [30]:
print(model) #full model architecture, all layers

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [31]:
print(model.config) #Model config: hidden size, num labels, dropout, etc.

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [32]:
# Training args
from transformers import TrainingArguments, BertModel,TrainingArguments, BertConfig, PreTrainedModel, Trainer, BertTokenizer, BertForSequenceClassification

training_args = TrainingArguments(
    output_dir="./bert_output_1",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
     report_to="none"
)

In [33]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [34]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

In [36]:
from transformers import BertModel, BertTokenizer, PreTrainedModel, Trainer, TrainingArguments, BertConfig
from transformers.modeling_outputs import SequenceClassifierOutput

In [37]:
class BertBinaryClassifier(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = nn.Linear(config.hidden_size, 2)  # Binary classification logits

        # Freeze all layers
        for param in self.bert.parameters():
            param.requires_grad = False

        # Unfreeze last 2 layers
        for layer in self.bert.encoder.layer[-2:]:
            for param in layer.parameters():
                param.requires_grad = True

        # Unfreeze pooler
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        # Classifier should be trainable
        for param in self.classifier.parameters():
            param.requires_grad = True

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits
        )

In [40]:
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)
model = BertBinaryClassifier(config)

training_args = TrainingArguments(
    output_dir="./bert_last2layers_finetune",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="no",
    report_to="none",  # Disable wandb etc.
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "architec

Step,Training Loss


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
def predict_text(text, model, tokenizer):
    # Preprocess input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Put model in eval mode
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()

    label_name = "Positive ✅" if predicted_class == 1 else "Negative ❌"
    print(f"Text: {text}")
    print(f"Prediction: {label_name} (Confidence: {confidence:.2f})")
    return predicted_class, confidence